In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip uninstall openpyxl
!pip install openpyxl

     |████████████████████████████████| 132 kB 9.2 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219295 sha256=ce2babecf66ff8b8d206a63f798a847a9cb8f7585791689ae08ad255f06062b7
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
     |████████████████████████████████| 11.3 MB 9.9 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 7.7 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
import missingno
from sklearn.preprocessing import LabelEncoder
import re
import matplotlib as mpl

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 모델 학습시키기

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/조현정/data(2016)_2.csv',index_col=0)
df1.rename(columns={'일자':'납기일자','prod_name':'제품명','prediction':'예측중량','sold_quant':'판매수량'},inplace=True)
df1 = pd.pivot_table(df1,
                    index = ['제품명','납기일자'],
                    aggfunc='sum')
df1.reset_index(inplace=True)
df1.drop('거래처코드',axis=1,inplace=True)
df1 = df1[df1.납기일자>'2017-12-31']
df1

,제품명,납기일자,판매수량
327,AE,2018-01-03,6080.0
328,AE,2018-01-04,1850.0
329,AE,2018-01-06,1240.0
330,AE,2018-01-11,3070.0
331,AE,2018-01-12,630.0
...,...,...,...
16258,SRE-200,2020-08-14,9900.0
16259,SRE-200,2020-10-09,10760.0
16260,SRE-200,2020-12-04,15000.0
16261,SRE-200,2021-01-29,8610.0


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/배송이/order_quant/중장기수주예측_1801_2106.csv',index_col=0)
df[df['납기일자']<'2021-05-01']
df

,납기일자,예측중량,제품명
0,2018-01-01,57576.0,PEMA-HR1000
1,2018-01-02,68393.0,PEMA-HR1000
2,2018-01-03,28731.0,PEMA-HR1000
3,2018-01-04,0.0,PEMA-HR1000
4,2018-01-05,79399.0,PEMA-HR1000
...,...,...,...
176,2021-06-26,0.0,PEMA-SPR
177,2021-06-27,0.0,PEMA-SPR
178,2021-06-28,0.0,PEMA-SPR
179,2021-06-29,0.0,PEMA-SPR


In [ ]:
df2 = pd.merge(df1,df,on=['납기일자','제품명'], how='left')
df2

,제품명,납기일자,판매수량,예측중량
0,AE,2018-01-03,6080.0,227.0
1,AE,2018-01-04,1850.0,167.0
2,AE,2018-01-06,1240.0,8646.0
3,AE,2018-01-11,3070.0,289.0
4,AE,2018-01-12,630.0,3074.0
...,...,...,...,...
10130,SRE-200,2020-08-14,9900.0,252.0
10131,SRE-200,2020-10-09,10760.0,252.0
10132,SRE-200,2020-12-04,15000.0,252.0
10133,SRE-200,2021-01-29,8610.0,311.0


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/증식완료(orderquant포함).csv',index_col=0)

df.rename(columns={'일자':'납기일자','prod_name':'제품명','prediction':'예측중량','sold_quant':'판매수량'},inplace=True)
df = pd.pivot_table(df,
                    index = ['제품명','납기일자'],
                    aggfunc='sum')
df.reset_index(inplace=True)
df['납기일자']=pd.to_datetime(df['납기일자'])
df['month']=df['납기일자'].dt.month


number = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/조현정/자료조사.xlsx')
number['month']=pd.to_datetime(number['month'],format='%Y/%m')
number['month']=number['month'].dt.month
df_number=pd.DataFrame(number.groupby('month')['가동률지수'].mean())
df_number.reset_index(inplace=True)


# df_stock = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/조현정/stock_data.xlsx',sheet_name='한일시멘트',index_col=0)
# df_stock['날짜'] = pd.to_datetime(df_stock.날짜)
# df_stock['month'] = df_stock.날짜.dt.month
# df_stock['year'] = df_stock.날짜.dt.year
# df_stock = df_stock[['month','year','종가']]
# df_stock = pd.DataFrame(df_stock.groupby('month')['종가'].mean())

import pickle5 as pickle
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/날씨_주별.pkl', "rb") as fh:
  weather = pickle.load(fh)
weather_대전 = weather[weather['지역']=='대전광역시']
# weather_대전 = weather[weather['일자']<'2021-01-01']
grouped_ta = weather_대전.groupby(weather_대전.일자.dt.month)['기온'].mean()
grouped_rn = weather_대전.groupby(weather_대전.일자.dt.month)['강수량'].mean()



# 월평균 가동률지수
def number(month):
  if month in [1]: return df_number['가동률지수'][0]
  elif month in [2]: return df_number['가동률지수'][1]
  elif month in [3]: return df_number['가동률지수'][2]
  elif month in [4]: return df_number['가동률지수'][3]
  elif month in [5]: return df_number['가동률지수'][4]
  elif month in [6]: return df_number['가동률지수'][5]
  elif month in [7]: return df_number['가동률지수'][6]
  elif month in [8]: return df_number['가동률지수'][7]
  elif month in [9]: return df_number['가동률지수'][8]
  elif month in [10]: return df_number['가동률지수'][9]
  elif month in [11]: return df_number['가동률지수'][10]
  elif month in [12]: return df_number['가동률지수'][11]
df['가동률지수']=df.month.apply(number)



# 대전광역시 월평균 기온
def m_temp(month):
    if month in [1]: return grouped_ta.iloc[0]
    elif month in [2]: return grouped_ta.iloc[1]	
    elif month in [3]: return grouped_ta.iloc[2]
    elif month in [4]: return grouped_ta.iloc[3]
    elif month in [5]: return grouped_ta.iloc[4]
    elif month in [6]: return grouped_ta.iloc[5]
    elif month in [7]: return grouped_ta.iloc[6]
    elif month in [8]: return grouped_ta.iloc[7]
    elif month in [9]: return grouped_ta.iloc[8]
    elif month in [10]: return grouped_ta.iloc[9]
    elif month in [11]: return grouped_ta.iloc[10]
    elif month in [12]: return grouped_ta.iloc[11]
df['m_temp'] = df.month.apply(m_temp)


# 대전광역시 월평균 강수량
def m_rn(month):
    if month in [1]: return grouped_rn.iloc[0]
    elif month in [2]: return grouped_rn.iloc[1]	
    elif month in [3]: return grouped_rn.iloc[2]
    elif month in [4]: return grouped_rn.iloc[3]
    elif month in [5]: return grouped_rn.iloc[4]
    elif month in [6]: return grouped_rn.iloc[5]
    elif month in [7]: return grouped_rn.iloc[6]
    elif month in [8]: return grouped_rn.iloc[7]
    elif month in [9]: return grouped_rn.iloc[8]
    elif month in [10]: return grouped_rn.iloc[9]
    elif month in [11]: return grouped_rn.iloc[10]
    elif month in [12]: return grouped_rn.iloc[11]
df['m_rn'] = df.month.apply(m_rn)


from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df[['가동률지수']] = scaler.fit_transform(df[['가동률지수']])

df['log_예측중량'] = np.log(df.예측중량)
df['log_판매수량'] = np.log(df.판매수량)


temp = df.drop(['예측중량','판매수량','month','납기일자'], axis =1)

bdf = pd.get_dummies(temp)



from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score, accuracy_score




xgb_reg = XGBRegressor()

lgbm_reg = LGBMRegressor()

models = [lr_reg,rf_reg,dt_reg,xgb_reg,lgbm_reg]


  
def get_rmse_pred(preds):
    for key in preds.keys():
        pred_value = preds[key]
        mse = mean_squared_error(y_test , pred_value)
        rmse = np.sqrt(mse)
        print('{0} 모델의 RMSLE: {1}'.format(key, rmse))


# 독립변수, 종속변수 분리
y_t_df = bdf['log_판매수량'] # 종속변수
X_t_df = bdf.drop('log_판매수량', axis = 1) # 독립변수

X_train, X_test, y_train, y_test = train_test_split(X_t_df, y_t_df, test_size = 0.2,
                                                   random_state = 0)


lr_reg = LinearRegression()

rf_reg = RandomForestRegressor()

dt_reg = DecisionTreeRegressor()

xgb_reg = XGBRegressor(n_estimators=5000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)
lgbm_reg = LGBMRegressor(n_estimators=5000, learning_rate=0.05, num_leaves=4, 
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)

xgb_reg.fit(X_train, y_train)
lgbm_reg.fit(X_train, y_train)

xgb_model1 = xgb_reg.fit(X_train, y_train)
lgbm_model1 = lgbm_reg.fit(X_train, y_train)


# import xgboost as xgb
# import ligbtgbm as lgbm
# save in JSON format
# xgb_model1.save_model("/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/xgb1.json")
# lgbm_model1.save_model("/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/lgbm1.json")




# # save the model to disk
# filename_xgb = '/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/xgb1.sav'
# filename_lgbm = '/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/lgbm1.sav'
# pickle.dump(xgb_model1, open(filename_xgb, 'wb'))
# pickle.dump(lgbm_model1, open(filename_lgbm, 'wb'))
 
# # save model
# joblib.dump(xgb_model1, '/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/xgb1.pkl')
# joblib.dump(lgbm_model1, '/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/lgb1.pkl')

xgb_pred = xgb_reg.predict(X_test)
lgbm_pred = lgbm_reg.predict(X_test)
lr_pred = lr_reg.predict(X_test)
rf_pred = lr_reg.predict(X_test)
dt_pred = lr_reg.predict(X_test)

pred = 0.5 * xgb_pred + 0.5 * lgbm_pred
preds = {'최종 혼합': pred,
         'XGBM': xgb_pred,
         'LGBM': lgbm_pred}
#최종 혼합 모델, 개별모델의 RMSLE 값 출력        
get_rmse_pred(preds)
print('혼합: Variance score : {0:.3f}'.format(r2_score(y_test,pred)))
print('XGBM: Variance score : {0:.3f}'.format(r2_score(y_test,xgb_pred)))
print('LGBM: Variance score : {0:.3f}'.format(r2_score(y_test,lgbm_pred)))
print('Linear Regression Variance score : {0:.3f}'.format(r2_score(y_test,lr_pred)))
print('Random Forest Variance score : {0:.3f}'.format(r2_score(y_test,rf_pred)))
print('Decision Tree Regression Variance score : {0:.3f}'.format(r2_score(y_test,dt_pred)))

[03:27:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:27:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


### 학습된 모델 사용해서 예측 테이블 만들기

In [ ]:
from sklearn.externals import joblib
import pickle5 as pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
# with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/중장기수주예측.pkl', "rb") as fh:
#   중장기수주예측 = pickle.load(fh)
# 중장기수주예측['납기일자'] = pd.to_datetime(중장기수주예측['납기일자'])
# # std_date = datetime.today().date()-timedelta(days=1) # today -> 실제사용시
# std_date = datetime.strptime("2021-04-04", "%Y-%m-%d") # 오늘이 21/4/5 이라고 가정
# 중장기수주예측 = 중장기수주예측[(중장기수주예측['납기일자']>std_date) & (중장기수주예측['납기일자']<std_date+relativedelta(months=3))]
# 중장기수주예측['month']=중장기수주예측['납기일자'].dt.month


중장기수주예측 = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/중장기수주예측.csv',index_col=0)
중장기수주예측['납기일자'] = pd.to_datetime(중장기수주예측['납기일자'])
# std_date = datetime.today().date()-timedelta(days=1) # today -> 실제사용시
std_date = datetime.strptime("2021-04-04", "%Y-%m-%d") # 오늘이 21/4/5 이라고 가정
중장기수주예측 = 중장기수주예측[(중장기수주예측['납기일자']>std_date) & (중장기수주예측['납기일자']<std_date+relativedelta(months=3))]
중장기수주예측['month']=중장기수주예측['납기일자'].dt.month

number = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/조현정/자료조사.xlsx')
number['month']=pd.to_datetime(number['month'],format='%Y/%m')
number['month']=number['month'].dt.month
df_number=pd.DataFrame(number.groupby('month')['가동률지수'].mean())
df_number.reset_index(inplace=True)


# df_stock = pd.read_excel('/content/drive/MyDrive/5조_스마트팩토리/조현정/stock_data.xlsx',sheet_name='한일시멘트',index_col=0)
# df_stock['날짜'] = pd.to_datetime(df_stock.날짜)
# df_stock['month'] = df_stock.날짜.dt.month
# df_stock['year'] = df_stock.날짜.dt.year
# df_stock = df_stock[['month','year','종가']]
# df_stock = pd.DataFrame(df_stock.groupby('month')['종가'].mean())


with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/날씨_주별.pkl', "rb") as fh:
  weather = pickle.load(fh)
weather_대전 = weather[weather['지역']=='대전광역시']
weather_대전 = weather[weather['일자']<'2021-01-01']
grouped_ta = weather_대전.groupby(weather_대전.일자.dt.month)['기온'].mean()
grouped_rn = weather_대전.groupby(weather_대전.일자.dt.month)['강수량'].mean()



# 월평균 가동률지수
def number(month):
  if month in [1]: return df_number['가동률지수'][0]
  elif month in [2]: return df_number['가동률지수'][1]
  elif month in [3]: return df_number['가동률지수'][2]
  elif month in [4]: return df_number['가동률지수'][3]
  elif month in [5]: return df_number['가동률지수'][4]
  elif month in [6]: return df_number['가동률지수'][5]
  elif month in [7]: return df_number['가동률지수'][6]
  elif month in [8]: return df_number['가동률지수'][7]
  elif month in [9]: return df_number['가동률지수'][8]
  elif month in [10]: return df_number['가동률지수'][9]
  elif month in [11]: return df_number['가동률지수'][10]
  elif month in [12]: return df_number['가동률지수'][11]
중장기수주예측['가동률지수']=중장기수주예측.month.apply(number)



# 대전광역시 월평균 기온
def m_temp(month):
    if month in [1]: return grouped_ta.iloc[0]
    elif month in [2]: return grouped_ta.iloc[1]	
    elif month in [3]: return grouped_ta.iloc[2]
    elif month in [4]: return grouped_ta.iloc[3]
    elif month in [5]: return grouped_ta.iloc[4]
    elif month in [6]: return grouped_ta.iloc[5]
    elif month in [7]: return grouped_ta.iloc[6]
    elif month in [8]: return grouped_ta.iloc[7]
    elif month in [9]: return grouped_ta.iloc[8]
    elif month in [10]: return grouped_ta.iloc[9]
    elif month in [11]: return grouped_ta.iloc[10]
    elif month in [12]: return grouped_ta.iloc[11]
중장기수주예측['m_temp'] = 중장기수주예측.month.apply(m_temp)


# 대전광역시 월평균 강수량
def m_rn(month):
    if month in [1]: return grouped_rn.iloc[0]
    elif month in [2]: return grouped_rn.iloc[1]	
    elif month in [3]: return grouped_rn.iloc[2]
    elif month in [4]: return grouped_rn.iloc[3]
    elif month in [5]: return grouped_rn.iloc[4]
    elif month in [6]: return grouped_rn.iloc[5]
    elif month in [7]: return grouped_rn.iloc[6]
    elif month in [8]: return grouped_rn.iloc[7]
    elif month in [9]: return grouped_rn.iloc[8]
    elif month in [10]: return grouped_rn.iloc[9]
    elif month in [11]: return grouped_rn.iloc[10]
    elif month in [12]: return grouped_rn.iloc[11]
중장기수주예측['m_rn'] = 중장기수주예측.month.apply(m_rn)


def minus(예측중량):
  if 예측중량 < 0:
    return 0
  else:
    return 예측중량
중장기수주예측['예측중량'] = 중장기수주예측.예측중량.apply(minus)


from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
중장기수주예측[['가동률지수']] = scaler.fit_transform(중장기수주예측[['가동률지수']])

중장기수주예측['log_예측중량'] = np.log(중장기수주예측.예측중량)
중장기수주예측['log_예측중량'] = 중장기수주예측.log_예측중량.apply(minus)

x_test_test = 중장기수주예측.drop(['예측중량','month','납기일자'], axis =1)

x_test_test = pd.get_dummies(x_test_test)


# load model
new_xgb_model = joblib.load('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/xgb1.pkl')
new_lgbm_model = joblib.load('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/lgb1.pkl')



xgb_pred = new_xgb_model.predict(x_test_test)
lgbm_pred = new_lgbm_model.predict(x_test_test)

pred_혼합 = 0.5 * xgb_pred + 0.5 * lgbm_pred
pred = pd.DataFrame(pred_혼합,columns=['생산량'])
pred['생산량'] = np.ceil(np.expm1(pred.생산량))
중장기수주예측 = 중장기수주예측[['납기일자','제품명']]
중장기수주예측.reset_index(drop=True,inplace=True)
생판계획 = pd.concat([중장기수주예측,pred],axis=1)

pd.set_option('display.max_rows',None)
생판계획.sort_values(by='납기일자',ascending=True,inplace=True)
생판계획.reset_index(drop=True,inplace=True)
생판계획

[19:54:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,납기일자,제품명,생산량
0,2021-04-05,PEMA-HR1000,22381.0
1,2021-04-05,AE,22546.0
2,2021-04-05,CSA5000F,23974.0
3,2021-04-05,PEMA-PCM2000B,23176.0
4,2021-04-05,PEMA-CR1000,23817.0
5,2021-04-05,PEMA-PCR3000N,21415.0
6,2021-04-05,PEMA-PR1000,20661.0
7,2021-04-05,SRE-200,20919.0
8,2021-04-05,PEMA-SN400,23916.0
9,2021-04-05,SRE-110,22639.0


In [ ]:
생판계획.to_csv('/content/drive/MyDrive/5조_스마트팩토리/배송이/생판계획.csv')